In [1]:
from curated_atlas_query_py import get_metadata, get_anndata
import session_info

In [88]:
import itertools
import os
import re
import shutil
import sys
from pathlib import Path
from typing import Iterable, Literal

import anndata as ad
import numpy as np
import scipy.sparse as sp
import duckdb
import pandas as pd
import requests
from appdirs import user_cache_dir
from tqdm import tqdm

REMOTE_URL = "https://object-store.rc.nectar.org.au/v1/AUTH_06d6e008e3e642da99d806ba3ea629c5"
ASSAY_URL = "{}/cellNexus-anndata".format(REMOTE_URL)
METADATA_URL = "{}/cellNexus-metadata/metadata.1.0.10.parquet".format(REMOTE_URL)
MIN_EXPECTED_SIZE = 5000000

assay_map = {"counts": "counts", "cpm": "cpm"}

def is_parquet_valid(parquet_file):
    try:
        conn = duckdb.connect()
        conn.from_parquet(str(parquet_file))  # Try reading
        return True  # File is valid
    except Exception as e:
        print(f"Parquet file is corrupt: {e}")
        return False  # File is corrupt
        
def _get_default_cache_dir() -> Path:
    return Path(user_cache_dir("cellNexusPy"))


# helper function to download file over http/https
def _sync_remote_file(full_url: str, output_file: Path):
    if not output_file.exists():
        output_dir = output_file.parent
        output_dir.mkdir(parents=True, exist_ok=True)
        print(f"Downloading {full_url} to {output_file}", file=sys.stderr)
        req = requests.get(full_url, stream=True, allow_redirects=True)
        req.raise_for_status()
        pbar = tqdm(total=int(req.headers.get("Content-Length", 0)))
        with pbar.wrapattr(req.raw, "read") as src, output_file.open("wb") as dest:
            shutil.copyfileobj(src, dest)

# function to get metadata
def get_metadata(
    parquet_url: str = METADATA_URL,
    cache_dir: os.PathLike[str] = _get_default_cache_dir(),
) -> tuple[duckdb.DuckDBPyConnection, duckdb.DuckDBPyRelation]:
    parquet_local = Path(cache_dir) / parquet_url.split("/")[-1]

    if not parquet_local.exists() or not is_parquet_valid(parquet_local):
        print("File is missing or corrupted. Re-downloading...")
        parquet_local.unlink(missing_ok=True)  # Delete the corrupted file
        _sync_remote_file(parquet_url, parquet_local)  # Re-download
    
    _sync_remote_file(parquet_url, parquet_local)
    conn = duckdb.connect()
    return conn, conn.from_parquet(str(parquet_local))

def sync_assay_files(
    url: str = ASSAY_URL,
    cache_dir: Path = _get_default_cache_dir(),
    subdirs: Iterable[str] = [],
    atlases: Iterable[str] = [],
    files: Iterable[str] = [],
):
    for atlas in atlases:
        for subdir in subdirs:
            for file in files:
                sub_url = f"{url}/{atlas}/{subdir}/{file}"
                output_filepath = cache_dir / atlas / subdir / file

                if not output_filepath.exists() or os.path.getsize(output_filepath) < MIN_EXPECTED_SIZE:
                    _sync_remote_file(sub_url, output_filepath)

                yield subdir, output_filepath

def _sort_anndata(data,anndata,cells,features):
    anndata.obs.index = anndata.obs.index.astype(str)
    cell_ids = cells.astype(str)
    pattern = '|'.join(re.escape(s) for s in cell_ids)
    mask = anndata.obs.index.str.contains(pattern, regex=True)

    anndata = anndata[mask,features].copy()
    
    positions_per_cell = []
    
    for cid in cell_ids:
        pos = np.where(anndata.obs.index.str.contains(cid))[0]
        positions_per_cell.append(pos)

    ann = anndata[np.concatenate(positions_per_cell).tolist(),:].copy()
    ann.obs = data.fetchdf()

    return ann


def _filter_obs(data,aggregation,anndata,cells,features):
    if aggregation == "single_cell":
        ann = _sort_anndata(data,anndata,cells,features)
        return ann
    else:
        '''
        TODO:

        Add AnnData.obs filtering when pseudobulk or metacells are required
        '''
        ann = _sort_anndata(data,anndata,cells,features)
        return ann

def get_anndata(
    data: duckdb.DuckDBPyRelation,
    assays: Iterable[Literal["counts", "cpm"]] = ["counts", "cpm"],
    aggregation: Iterable[Literal["single_cell", "pseudobulk", "metacell"]] = ["single_cell", "pseudobulk", "metacell"],
    cache_directory: Path = _get_default_cache_dir(),
    repository: str = ASSAY_URL,
    features: Iterable[str] = slice(None, None, None),
) -> ad.AnnData:
    # error checking
    assert set(assays).issubset(set(assay_map.keys()))
    assert isinstance(cache_directory, Path), "cache_directory must be a Path"
    
    assays = set(assays)
    cache_directory.mkdir(exist_ok=True, parents=True)
    
        # all the files to retrieve for query
    if str(aggregation[0])=="metacell":
        aggregation = ["single_cell"] 
        data = data.filter("metacell_2 IS NOT NULL")
    
    files_to_read = (
        data.project("file_id_cellNexus_"+str(aggregation[0])).distinct().fetchdf()["file_id_cellNexus_"+str(aggregation[0])]
    )
    cells = data.project('"cell_id"').distinct().fetchdf()["cell_id"]
    atlases = data.project('"atlas_id"').distinct().fetchdf()["atlas_id"]
        # mapping requested assay to subdirectory name
    subdirs = [assay_map[a] for a in assays]                                                                                                                         
    
    synced = sync_assay_files(
        url=repository, cache_dir=cache_directory, atlases=atlases, subdirs=subdirs, files=files_to_read
    )
    
    ad_per_assay: dict[str, ad.AnnData] = {}
        # Concatenate files per assay
    for assay_name, files in itertools.groupby(synced, key=lambda x: x[0]):
        ads = [ad.read_h5ad(file[1]) for file in files]
        ad_per_assay[assay_name] = ad.concat(ads, index_unique="-")
    
    main_assay_key = (
        "counts" if "counts" in ad_per_assay else next(iter(ad_per_assay.keys()))
    )
    
    ann = ad_per_assay.pop(main_assay_key)
        # Attach the additional assays
    for key, anndata in ad_per_assay.items():
        ann.layers[key] = anndata.X
    ann = _filter_obs(data,aggregation[0],ann,cells,features)
    return ann

In [6]:
session_info.show()

/home/juan/anaconda3/envs/cellnexuspy/lib/python3.10/site-packages/session_info/main.py:213: UserWarning: The '__version__' attribute is deprecated and will be removed in MarkupSafe 3.1. Use feature detection, or `importlib.metadata.version("markupsafe")`, instead.
  mod_version = _find_version(mod.__version__)


In [5]:
conn, table = get_metadata()
table

┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────┬────────────────────┬──────────────────────────────────┬─────────────────────────────────┬────────────────────────────┬──────────────────────────────────┬───────────┬────────────────────────┬────────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬───────────────────┬─────────────────────────┬────────────────────────────────────┬──────────────────────────────┬──────────

In [6]:
table.aggregate("tissue, cell_type_unified_ensemble, COUNT(*) as n", group_expr="tissue, cell_type_unified_ensemble")

┌───────────────────────────────────┬────────────────────────────┬─────────┐
│              tissue               │ cell_type_unified_ensemble │    n    │
│              varchar              │          varchar           │  int64  │
├───────────────────────────────────┼────────────────────────────┼─────────┤
│ lung                              │ cd8 tcm                    │   26685 │
│ lymph node                        │ macrophage                 │    5142 │
│ lung                              │ cytotoxic                  │   10029 │
│ lung                              │ cd14 mono                  │  107395 │
│ brain                             │ nk                         │    3300 │
│ lung                              │ monocytic                  │   47557 │
│ brain                             │ pericyte                   │    1459 │
│ brain                             │ cd4 th2 em                 │     367 │
│ lymph node                        │ pdc                        │     411 │

In [7]:
query = table.filter("""
    self_reported_ethnicity = 'African'
    AND assay LIKE '%10%'
    AND tissue = 'lung parenchyma'
    AND cell_type LIKE '%CD4%'
""")
query

┌────────────────────────────────────────────────────────────────┬──────────────────────────────────────┬────────────────────┬──────────────────────────────────────────┬─────────────────────────────────┬────────────────────────────┬──────────────────────────────────┬───────────┬────────────────────────┬────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────────────────────────────────┬──────────────────────────────────────┬───────────────────┬─────────────────────────┬────────────────────────────────────┬─────────┬──────────────────────────┬───────────────────────────────────────────────────────────────────────────────┬───────────────┬──────────────────────────────────────────────────────────────────────────────┬─────

# COUNTS EXAMPLE

In [75]:
ann = get_anndata(query, assays=["counts"],aggregation=["single_cell"])
ann

# CPM EXAMPLE

In [79]:
ann = get_anndata(query, assays=["cpm"],aggregation=["single_cell"])
ann

  0%|                                               | 0/2226408 [00:00<?, ?it/s]
0it [00:00, ?it/s]
64.0kB [00:00, 88.3kB/s]
128kB [00:01, 137kB/s]  
192kB [00:01, 185kB/s]
256kB [00:01, 237kB/s]
448kB [00:01, 480kB/s]
512kB [00:01, 412kB/s]
768kB [00:02, 585kB/s]
1.06MB [00:02, 648kB/s]
1.38MB [00:02, 783kB/s]
1.69MB [00:03, 881kB/s]
2.12MB [00:03, 645kB/s]
  0%|                                               | 0/3512397 [00:00<?, ?it/s]
0it [00:00, ?it/s]
64.0kB [00:00, 112kB/s]
128kB [00:00, 157kB/s] 
192kB [00:01, 182kB/s]
448kB [00:01, 426kB/s]
896kB [00:01, 768kB/s]
1.75MB [00:02, 1.48MB/s]
3.35MB [00:02, 1.39MB/s]
  0%|                                               | 0/2908364 [00:00<?, ?it/s]
0it [00:00, ?it/s]
64.0kB [00:00, 112kB/s]
128kB [00:00, 158kB/s] 
192kB [00:01, 179kB/s]
448kB [00:01, 416kB/s]
832kB [00:01, 706kB/s]
1.25MB [00:02, 962kB/s]
1.69MB [00:02, 1.13MB/s]
2.12MB [00:02, 1.24MB/s]
2.77MB [00:03, 955kB/s] 
  0%|                                               | 0/

# PSEUDOBULK EXAMPLE

In [83]:
ann = get_anndata(query,aggregation=["pseudobulk"])
ann

AnnData object with n_obs × n_vars = 1800 × 56239
    obs: 'cell_id', 'dataset_id', 'observation_joinid', 'sample_id', 'cell_type', 'cell_type_ontology_term_id', 'sample_', 'assay', 'assay_ontology_term_id', 'cell_count', 'citation', 'collection_id', 'dataset_version_id', 'default_embedding', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'experiment___', 'explorer_url', 'feature_count', 'filesize', 'filetype', 'is_primary_data', 'mean_genes_per_cell', 'organism', 'organism_ontology_term_id', 'primary_cell_count', 'published_at', 'raw_data_location', 'revised_at', 'run_from_cell_id', 'sample_heuristic', 'schema_version', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'title', 'tombstone', 'url', 'x_approximate_distribution', 'X_umap1', 'X_umap2', 'age_days', 'tissue_groups', 'atlas_id', 'cell_annotat

# METACELL EXAMPLE

In [89]:
ann = get_anndata(query,aggregation=["metacell"])
ann

AnnData object with n_obs × n_vars = 1405 × 56239
    obs: 'cell_id', 'dataset_id', 'observation_joinid', 'sample_id', 'cell_type', 'cell_type_ontology_term_id', 'sample_', 'assay', 'assay_ontology_term_id', 'cell_count', 'citation', 'collection_id', 'dataset_version_id', 'default_embedding', 'development_stage', 'development_stage_ontology_term_id', 'disease', 'disease_ontology_term_id', 'donor_id', 'experiment___', 'explorer_url', 'feature_count', 'filesize', 'filetype', 'is_primary_data', 'mean_genes_per_cell', 'organism', 'organism_ontology_term_id', 'primary_cell_count', 'published_at', 'raw_data_location', 'revised_at', 'run_from_cell_id', 'sample_heuristic', 'schema_version', 'self_reported_ethnicity', 'self_reported_ethnicity_ontology_term_id', 'sex', 'sex_ontology_term_id', 'suspension_type', 'tissue', 'tissue_ontology_term_id', 'tissue_type', 'title', 'tombstone', 'url', 'x_approximate_distribution', 'X_umap1', 'X_umap2', 'age_days', 'tissue_groups', 'atlas_id', 'cell_annotat